## Description of what we are going to do

In [ ]:
import os
import geopandas
import pandas
import zipfile
import subprocess
import logging
from typing import Union, List, Dict, Tuple, Optional
from ipyleaflet import Map, GeoData, basemaps, LayersControl

In [ ]:
BASE_DIR = './../../datasets/raw'

def extract_wdpa_data(wdpa_zip_file: str) -> int:
    """
    Extract WDPA data from a zip file.
    """
    try:
        with zipfile.ZipFile(wdpa_zip_file, 'r') as zip_ref:
            zip_ref.extractall(BASE_DIR)
        return 0
    except:
        logging.error('Could not extract WDPA data from zip file.')
        return 1

def load_wdpa_data(dir_path: str) -> Union[int, geopandas.GeoDataFrame]:
    """
    Load WDPA data from a csv file.
    """
    try:
        f: List = []
        for (dirpath, dirnames, filenames) in os.walk(dir_path):
            for filename in filenames:
                f.append(f'{dirpath}/{filename}')

        gdf = pandas.concat([
            geopandas.read_file(file)
            for file in f
        ]).pipe(geopandas.GeoDataFrame)
        return gdf
    except:
        logging.error('Could not load WDPA data from csv file.')
        return 1

def simplifyMapshaper(data_path: str, format: str = "shp") -> Union[int, geopandas.GeoDataFrame]:
    """
    Simplify geometry of a GeoDataFrame using mapshaper.
    """
    try:
        result = subprocess.run(["mapshaper-xl", "16gb", f"-i {data_path}/*.{format} snap", 
        "-simplify 20% planar keep-shapes", "-filter-islands min-vertices=3 min-area=10000m2 remove-empty",
        "-filter-slivers min-area=10000m2 remove-empty",
        "-clean rewind", "-o format=geojson force ndjson"], check=True)
        
        return load_wdpa_data()
    
    except Exception as e:
        logging.error(e)
        return 1

def mapRender(geo_data: geopandas.GeoDataFrame) -> Map:
    """
    Render a map with the given data.
    """
    try:
        return Map(
            center=[0, 0],
            zoom=2,
            basemap=basemaps.Esri.WorldImagery,
            layers=[
                GeoData(
                    data=geo_data,
                    name='WDPA',
                    style={
                        'color': '#ff0000',
                        'fillOpacity': 0.5
                    }
                )
            ],
            layers_control=True,
            controls=[
                LayersControl()
            ]
        )
    except:
        logging.error('Could not render map.')
        return 1